In [ ]:
!pip install tensorflow==2.2.0
!pip install keras
!pip install kerassurgeon

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerassurgeon.identify import get_apoz;
from kerassurgeon.operations import delete_layer, insert_layer, delete_channels;
from kerassurgeon import Surgeon
from kerassurgeon import utils

# The Speck cipher and data generation algorithms #
Taken from https://github.com/agohr/deep_speck/blob/master/speck.py

In [ ]:
import numpy as np
from os import urandom

def WORD_SIZE():
    return(16);

def ALPHA():
    return(7);

def BETA():
    return(2);

MASK_VAL = 2 ** WORD_SIZE() - 1;

def shuffle_together(l):
    state = np.random.get_state();
    for x in l:
        np.random.set_state(state);
        np.random.shuffle(x);

def rol(x,k):
    return(((x << k) & MASK_VAL) | (x >> (WORD_SIZE() - k)));

def ror(x,k):
    return((x >> k) | ((x << (WORD_SIZE() - k)) & MASK_VAL));

def enc_one_round(p, k):
    c0, c1 = p[0], p[1];
    c0 = ror(c0, ALPHA());
    c0 = (c0 + c1) & MASK_VAL;
    c0 = c0 ^ k;
    c1 = rol(c1, BETA());
    c1 = c1 ^ c0;
    return(c0,c1);

def dec_one_round(c,k):
    c0, c1 = c[0], c[1];
    c1 = c1 ^ c0;
    c1 = ror(c1, BETA());
    c0 = c0 ^ k;
    c0 = (c0 - c1) & MASK_VAL;
    c0 = rol(c0, ALPHA());
    return(c0, c1);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);

def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

def check_testvector():
  key = (0x1918,0x1110,0x0908,0x0100)
  pt = (0x6574, 0x694c)
  ks = expand_key(key, 22)
  ct = encrypt(pt, ks)
  if (ct == (0xa868, 0x42f2)):
    print("Testvector verified.")
    return(True);
  else:
    print("Testvector not verified.")
    return(False);

#convert_to_binary takes as input an array of ciphertext pairs
#where the first row of the array contains the lefthand side of the ciphertexts,
#the second row contains the righthand side of the ciphertexts,
#the third row contains the lefthand side of the second ciphertexts,
#and so on
#it returns an array of bit vectors containing the same data
def convert_to_binary(arr):
  X = np.zeros((4 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(4 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

#takes a text file that contains encrypted block0, block1, true diff prob, real or random
#data samples are line separated, the above items whitespace-separated
#returns train data, ground truth, optimal ddt prediction
def readcsv(datei):
    data = np.genfromtxt(datei, delimiter=' ', converters={x: lambda s: int(s,16) for x in range(2)});
    X0 = [data[i][0] for i in range(len(data))];
    X1 = [data[i][1] for i in range(len(data))];
    Y = [data[i][3] for i in range(len(data))];
    Z = [data[i][2] for i in range(len(data))];
    ct0a = [X0[i] >> 16 for i in range(len(data))];
    ct1a = [X0[i] & MASK_VAL for i in range(len(data))];
    ct0b = [X1[i] >> 16 for i in range(len(data))];
    ct1b = [X1[i] & MASK_VAL for i in range(len(data))];
    ct0a = np.array(ct0a, dtype=np.uint16); ct1a = np.array(ct1a,dtype=np.uint16);
    ct0b = np.array(ct0b, dtype=np.uint16); ct1b = np.array(ct1b, dtype=np.uint16);
    
    #X = [[X0[i] >> 16, X0[i] & 0xffff, X1[i] >> 16, X1[i] & 0xffff] for i in range(len(data))];
    X = convert_to_binary([ct0a, ct1a, ct0b, ct1b]); 
    Y = np.array(Y, dtype=np.uint8); Z = np.array(Z);
    return(X,Y,Z);

#baseline training data generator
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

#real differences data generator
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  #generate keys
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);


# Evaluate the results #
Taken from https://github.com/agohr/deep_speck/blob/master/eval.py and slightly adapted.

In [ ]:
def evaluate(net,X,Y):
    
    Z = net.predict(X,batch_size=10000).flatten();
    Zbin = (Z > 0.5);
    
    #Compute the acc, tpr, tnr
    n = len(Z); 
    n0 = np.sum(Y==0); 
    n1 = np.sum(Y==1);
    
    acc = np.sum(Zbin == Y) / n;
    tpr = np.sum(Zbin[Y==1]) / n1;
    tnr = np.sum(Zbin[Y==0] == 0) / n0;
    
    return(acc, tpr, tnr);

# Conduct multiple evaluations #

In [ ]:
def multiple_evaluations(model, repetitions, num_rounds):
 
  #The accs, tprs, tnrs for all evaluation repetition
  accs = [];
  tprs = [];
  tnrs = [];
    
  #Evaluate multiple times and average the results 
  for i in range(0, repetitions):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);

    (acc, tpr, tnr) = evaluate(model, X_eval, Y_eval);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

# The depth-1/10 distinguisher implementation #
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted for running multiple trials. 

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.regularizers import l2, l1, l1_l2


def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

#Batch size
bs = 5000;


def make_resnet(num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=5, reg_param=0.0001, final_activation='sigmoid'):
  
  #Input and preprocessing layers
  inp = Input(shape=(num_blocks * word_size * 2,));
  rs = Reshape((2 * num_blocks, word_size))(inp);
  perm = Permute((2,1))(rs);
    
  #Block 1
  conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);
    
  #Blocks 2-i - residual blocks
  shortcut = conv0;
  for i in range(depth):
    conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
    conv1 = BatchNormalization()(conv1);
    conv1 = Activation('relu')(conv1);
    
    conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
    conv2 = BatchNormalization()(conv2);
    conv2 = Activation('relu')(conv2);
    shortcut = Add()([shortcut, conv2]);
    
  #Block 3
  flat1 = Flatten()(shortcut);
    
  dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);

  dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
    
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);

  model = Model(inputs=inp, outputs=out);

  return model;



def model_builder(depth):

  model = make_resnet(depth=depth);

  model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);
    
  return model;
  



def train_speck_distinguisher(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience= 3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    
    model.fit(X_train, Y_train, batch_size= bs, epochs= num_epochs, validation_data= (X_eval, Y_eval), callbacks=[lr, stop_early,])

    return model;


# Obtain filter/neuron indexes with the Average Percentage Of activations equal to Zero greater or equal to *percentage* #

In [ ]:
def get_indexes_with_apoz_gt(apoz_values, percentage):
  
  indexes =[];

  for i in range(0,len(apoz_values)):
    if apoz_values[i] >= percentage:
      indexes.append(i);
    
  #These indexes are used later to know which filter/neuron to prune
  return indexes;

# Repeat: Prune the model of # filters/neurons with an APoZ value >= *percentage*, train it, and evaluate the results

In [ ]:
def repeat_experiment_for_an_apoz_percentage(num_rounds, depth, num_experiment_trials, prune_above_apoz_percentage):

        #Store the accs, tprs, tnrs for all exeperiment repetitions for a speciffic APoZ cutoff (prune_above_apoz_percentage)
        accs =[];
        tprs =[];
        tnrs =[];
        
        #Store how many filters/neurons were pruned at each layer per experiment
        pruned_count = [[],[],[],[],[]];
       
        #Repeat the experiment multiple times 
        for trial in range(0, num_experiment_trials):
            
            #Train the depth-1 distinguisher
            X_train, Y_train = make_train_data(10**7,num_rounds);
            X_eval, Y_eval = make_train_data(10**6, num_rounds);
            initial_model = model_builder(depth=1);
            trained_model= train_speck_distinguisher(initial_model, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

            #Generate data for computing the APoZ values
            X, _ = make_train_data(10**6, num_rounds);

            #Store the names of the layer and its activation layer
            activation_layers =[];
            conv_or_dense_layers=[];
            
            for layer in trained_model.layers:
                
                if ('conv' in layer.name or 'dense' in layer.name):
                  conv_or_dense_layers.append(layer);
                
                if('activation' in  layer.name):
                  activation_layers.append(layer);

            #Instantiate the surgeon
            surgeon = Surgeon(trained_model);
            

            for i in range(0, len(activation_layers)):
              
              #Get the APoZ values for a speciffic layer
              act = get_apoz(trained_model, activation_layers[i], X);
              #Get the indexess of the filters/neurons to prune from the layer
              prune_indexes=get_indexes_with_apoz_gt(act, prune_above_apoz_percentage); 
              #Add job to prune the layer later 
              surgeon.add_job('delete_channels', conv_or_dense_layers[i], channels=prune_indexes);
              #Store how much was pruned
              pruned_count[i].append(len(prune_indexes)); 

              

            #Prune the model
            pruned_model =surgeon.operate();
            pruned_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc']);

            #Trained the pruned model
            X_train, Y_train = make_train_data(10**7,num_rounds);
            X_eval, Y_eval = make_train_data(10**6, num_rounds);
            trained_pruned_model = train_speck_distinguisher(pruned_model, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

            #Evaluate the pruned model
            (acc, tpr, tnr) = multiple_evaluations(trained_pruned_model,5,num_rounds);
            accs.append(acc);
            tprs.append(tpr);
            tnrs.append(tnr);

        print("Rounds: "+str(num_rounds) +" "+ "APOZ: "+str(prune_above_apoz_percentage));
        print("Average accuracy: "+str(np.mean(accs)) +" +/- "+ str(np.std(accs)));
        print("Average TPR: "+str(np.mean(tprs)) +" +/- "+ str(np.std(tprs)));
        print("Average TNR: "+str(np.mean(tnrs)) +" +/- "+ str(np.std(tnrs)));

        print("Average pruned filters at Conv1: "+ str(np.mean(pruned_count[0])));
        print("Average pruned filters at Conv2: "+ str(np.mean(pruned_count[1])));
        print("Average pruned filters at Conv3: "+ str(np.mean(pruned_count[2])));
        print("Average pruned neurons at Dense1: "+ str(np.mean(pruned_count[3])));
        print("Average pruned neurons at Dense2: "+ str(np.mean(pruned_count[4])));

        #Return the above-printed values
        return (np.mean(accs), np.mean(tprs), np.mean(tnrs), accs, tprs, tnrs, pruned_count[0], pruned_count[1], pruned_count[2], pruned_count[3], pruned_count[4]);


# Run the experiment #

In [ ]:
# num_rounds = 5, 6, 7, 8
# prune_above_apoz_percentage = 1, 0.9, 0.8, 0.7

(acc_avg, tpr_avg, tnr_avg, accs, tprs, tnrs, pc1, pc2, pc3, pd1, pd2) = repeat_experiment_for_an_apoz_percentage(num_rounds =  5, depth = 1, num_experiment_trials = 5, prune_above_apoz_percentage = 1);